Import Essential Libraries & Drives

In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
## Specify columns to import
cols_to_import = ['serial_num_field', 'activation_date',
       'activation_amount', 'last_trans_date', 'cur_balance']

In [ ]:
## Read .csv file and save as df
file_path = '/content/drive/My Drive/Colab Notebooks/3102511.cert_liabilities.20250102.1379843..csv'
data = pd.read_csv(file_path,usecols=cols_to_import)

<ipython-input-4-91d091a4364c>:3: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path)


Assess the Shape of the Data

In [ ]:
data.shape

(33881194, 9)

In [ ]:
## Filter out the $0 balances
nonzero_condition = data['cur_balance'] != 0
nonzero_data = data.loc[nonzero_condition]
nonzero_data.shape


(8384789, 9)

In [ ]:
## Preview the Data
nonzero_data.head()

,certificate_id,serial_num_field,activation_date,activation_amount,last_trans_date,cur_balance,store_ref,store_name,card_group
0,736865977,31066000-15776800,2020-06-11,43.21,2020-06-11,43.21,INCOMM,INCOMM / TEXAS ROADHOUSE,Incomm
1,736865978,31066000-15776801,2020-06-11,43.21,2020-06-11,43.21,INCOMM,INCOMM / TEXAS ROADHOUSE,Incomm
2,736865979,31066000-15776802,2020-06-11,43.21,2020-06-11,43.21,INCOMM,INCOMM / TEXAS ROADHOUSE,Incomm
15,736866023,31066000-15776846,2021-12-19,100.00,2022-03-22,14.90,INCOMM,INCOMM / TEXAS ROADHOUSE,Incomm
16,736866024,31066000-15776847,2021-12-19,100.00,2021-12-19,100.00,INCOMM,INCOMM / TEXAS ROADHOUSE,Incomm


In [ ]:
## Create a running total column using the "cur_balance" column
nonzero_data['running_total'] = nonzero_data['cur_balance'].cumsum()

<ipython-input-8-d77bde458b64>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nonzero_data['running_total'] = nonzero_data['cur_balance'].cumsum()


In [ ]:
## Assess all columns present in dataset
nonzero_data.columns

Index(['certificate_id', 'serial_num_field', 'activation_date',
       'activation_amount', 'last_trans_date', 'cur_balance', 'store_ref',
       'store_name', 'card_group', 'running_total'],
      dtype='object')

In [ ]:
## Trim the data to keep only essential columns
cols_to_keep = ['serial_num_field', 'activation_date',
       'activation_amount', 'last_trans_date', 'cur_balance','running_total']
trimmed_data = nonzero_data[cols_to_keep]

In [ ]:
## Analyze the overall sum of the running_total column
Running_total_sum = trimmed_data['running_total'].sum()
Current_bal_sum = trimmed_data['cur_balance'].sum()

print("The Running Total Sum is: $" + str(Running_total_sum))
print("The Current Balance Sum is: $" + str(Current_bal_sum))

The Running Total Sum is: $1112114457623069.9
The Current Balance Sum is: $257526487.49999997


In [ ]:
trimmed_data.reset_index(drop=True, inplace=True)
trimmed_data.head(10)

,serial_num_field,activation_date,activation_amount,last_trans_date,cur_balance,running_total
0,31066000-15776800,2020-06-11,43.21,2020-06-11,43.21,43.21
1,31066000-15776801,2020-06-11,43.21,2020-06-11,43.21,86.42
2,31066000-15776802,2020-06-11,43.21,2020-06-11,43.21,129.63
3,31066000-15776846,2021-12-19,100.00,2022-03-22,14.90,144.53
4,31066000-15776847,2021-12-19,100.00,2021-12-19,100.00,244.53
5,31066000-15776852,2021-12-18,50.00,2022-03-04,0.37,244.90
6,31066000-15776855,2022-10-01,50.00,2022-10-01,50.00,294.90
7,31066000-15776866,2022-09-17,30.00,2022-09-17,30.00,324.90
8,31066000-15776867,2022-10-02,25.00,2022-11-13,4.14,329.04
9,31066000-15776868,2023-04-15,50.00,2024-02-23,0.42,329.46


Chunk Data out Based on Parameters Selected from KPMG Auditor(s)

In [ ]:
## Create function to breakout dataset into subpopulations within KPMG Threshold
def breakout_df(df=trimmed_data, hit_threshold=1000):
  hit_condition = trimmed_data['running_total'] <= hit_threshold
  hit_data = trimmed_data.loc[hit_condition]
  row_count = len(hit_data)
  rows_needed = int(row_count) + 1
  df_save_name = 'INCOMM_breakout_hit_$'+str(hit_threshold)+'.csv'
  breakout_df = trimmed_data.head(rows_needed)
  breakout_df.to_csv(df_save_name,index=False)
  return breakout_df

In [ ]:
## Create KPMG Thresholds as an iterable list
kpmg_thresholds = [4264574.52,
                   46137919.07,
                   88011263.63,
                   129884608.19,
                   171757952.74,
                   213631297.30,
                   255504641.86]

for hit in kpmg_thresholds:
  breakout_df(trimmed_data,hit) ## Run breakout function